# Atividade 01 - Parte 01 - Inventário de emissões utilizando Python
Foi utilizado o capítulo 10.6 da AP42, pois a empresa fabrica MDF. https://www.epa.gov/sites/default/files/2020-10/documents/c10s0603.pdf

Fatores de emissão (FE) foram convertidos da seguinte maneira:

-Caso A: FE em lb/ODT = 0.5 kg/Mg = 0.35 kg/m3

Densidade do MDF = 700kg/m3 

-Caso B: FE em lb/MSF 3/4 = 0.26 kg/m3 

-Caso C: FE em lb/MSF = 0.0049 kg/m2 = 0.27 kg/m3

Espessura do MDF = 0.018m 

Taxa de atividade = 1.140 milhão m3/ano

In [8]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

#dados de entrada
ta = 1.14 # taxa de atividade em milhao m3/ano
path = "C:\ENS5173\Dados\FE.csv"
fe = pd.read_csv(path) # fatores de emissao em kg/m3 
print(fe)

                                              Source      PM  PM-10    NOX  \
0  Tube dryer, direct wood-fired, blowline blend,...  3.6400  0.560    NaN   
1              Hot press, UF resin (SCC 3-07-009-60)  0.0468  0.039  0.039   
2  Former with blowline blend, UF resin (SCC 3-07...     NaN    NaN    NaN   
3                           Sander (SCC 3-07-009-83)     NaN    NaN    NaN   
4                   Saw and hogger (SCC 3-07-009-84)     NaN    NaN    NaN   

        CO  VOC as propane  
0  1.40000        2.100000  
1  0.00884        0.208000  
2      NaN        0.023450  
3      NaN        0.001782  
4      NaN        0.035100  


In [9]:
# criando funcao para calcular as emissoes 

def emis(FE, TA):
    emis = pd.DataFrame()
    cols_numericas = FE.select_dtypes(np.float64).columns
    emis = FE[cols_numericas] * TA
    emis = emis.set_index(FE['Source'])
    return emis

emissoes = emis(fe, ta)
print(emissoes) 


                                                          PM    PM-10  \
Source                                                                  
Tube dryer, direct wood-fired, blowline blend, ...  4.149600  0.63840   
Hot press, UF resin (SCC 3-07-009-60)               0.053352  0.04446   
Former with blowline blend, UF resin (SCC 3-07-...       NaN      NaN   
Sander (SCC 3-07-009-83)                                 NaN      NaN   
Saw and hogger (SCC 3-07-009-84)                         NaN      NaN   

                                                        NOX        CO  \
Source                                                                  
Tube dryer, direct wood-fired, blowline blend, ...      NaN  1.596000   
Hot press, UF resin (SCC 3-07-009-60)               0.04446  0.010078   
Former with blowline blend, UF resin (SCC 3-07-...      NaN       NaN   
Sander (SCC 3-07-009-83)                                NaN       NaN   
Saw and hogger (SCC 3-07-009-84)                  